In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import glob
import pickle
import helper_functions_06b as hf

In [2]:
DATE = ['20230303', '20230305']
SESSION = ['SM_Lek1', 'SE_Lek1']
DRONE = ['P1D1', 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/TerritoryDetection2023'

In [3]:
# Thresholds for translation, rotation, and scale
thresholds = {
    'T_translation': 30,  # e.g., 30 pixels
    'T_rotation': 20,     # e.g., 20 degrees
    'T_scale': 0.1         # e.g., 10% change
}

In [ ]:
for date in DATE:
    for session in SESSION:
        for drone in DRONE:
            print(date, session, drone)
            
            internal_path = tracking_directory + '/' + date + '/' + session + '/' + drone
            files = glob.glob(internal_path + '/' + date + '_' + session + '_' + drone + '*_tracked.csv')
            
            for file in sorted(files):
                df = pd.read_csv(file)
                
                df['x'] = df['bb_left'] + df['bb_width']/2
                df['y'] = df['bb_top'] + df['bb_height']/2
                df['idx'] = df['track_id']

                df = df.loc[:,['frame', 'x', 'y', 'idx']]
                
                # Step 1: Identify anchor frames
                anchor_frames = hf.identify_anchor_frames(df, thresholds)

                # Step 2: Find the best anchor for each frame
                df = hf.find_best_anchor_for_each_frame_parallel(df, anchor_frames)

                # Step 3: Apply homography using the best anchor for each frame
                df, homography_matrices, rms_errors = hf.apply_homography_to_best_anchor(df)

                df.to_csv(file[:-17] + '_Anchored.csv', index=False, mode='w')
                with open(file[:-17] + '_homographies.pkl', 'wb') as f:
                    pickle.dump(homography_matrices, f)

20230303 SM_Lek1 P1D1
Initial identification of anchors takes  15.408824920654297
Finding the best anchors took 215.9923849105835
Applying homography to the best anchors takes  147.76933193206787
Initial identification of anchors takes  14.509182214736938
Finding the best anchors took 216.2675850391388
Applying homography to the best anchors takes  146.39184737205505
Initial identification of anchors takes  12.6510488986969
Finding the best anchors took 37.56800365447998
Applying homography to the best anchors takes  150.70154404640198
Initial identification of anchors takes  18.70102310180664
Finding the best anchors took 128.69403076171875
Applying homography to the best anchors takes  150.20197200775146
Initial identification of anchors takes  12.841546058654785
Finding the best anchors took 44.47850489616394
Applying homography to the best anchors takes  152.98036313056946
Initial identification of anchors takes  1.9609227180480957
Finding the best anchors took 3.5390119552612305
A

Not enough matching points for homography between frame 3307 and frame 3812.
Not enough matching points for homography between frame 3307 and frame 3813.
Not enough matching points for homography between frame 3307 and frame 3814.
Not enough matching points for homography between frame 3307 and frame 3815.
Not enough matching points for homography between frame 3307 and frame 3816.
Not enough matching points for homography between frame 3307 and frame 3817.
Not enough matching points for homography between frame 3307 and frame 3818.
Not enough matching points for homography between frame 3307 and frame 3819.
Not enough matching points for homography between frame 3307 and frame 3820.
Not enough matching points for homography between frame 3307 and frame 3821.
Not enough matching points for homography between frame 3307 and frame 3822.
Not enough matching points for homography between frame 3307 and frame 3823.
Not enough matching points for homography between frame 3307 and frame 3824.

Not enough matching points for homography between frame 3307 and frame 4069.
Not enough matching points for homography between frame 3307 and frame 4070.
Not enough matching points for homography between frame 3307 and frame 4071.
Not enough matching points for homography between frame 3307 and frame 4072.
Not enough matching points for homography between frame 3307 and frame 4073.
Not enough matching points for homography between frame 3307 and frame 4074.
Not enough matching points for homography between frame 3307 and frame 4075.
Not enough matching points for homography between frame 3307 and frame 4076.
Not enough matching points for homography between frame 3307 and frame 4077.
Not enough matching points for homography between frame 3307 and frame 4078.
Not enough matching points for homography between frame 3307 and frame 4079.
Not enough matching points for homography between frame 3307 and frame 4080.
Not enough matching points for homography between frame 3307 and frame 4081.

Not enough matching points for homography between frame 3307 and frame 4310.
Not enough matching points for homography between frame 3307 and frame 4311.
Not enough matching points for homography between frame 3307 and frame 4312.
Not enough matching points for homography between frame 3307 and frame 4313.
Not enough matching points for homography between frame 3307 and frame 4314.
Not enough matching points for homography between frame 3307 and frame 4315.
Not enough matching points for homography between frame 3307 and frame 4316.
Not enough matching points for homography between frame 3307 and frame 4317.
Not enough matching points for homography between frame 3307 and frame 4318.
Not enough matching points for homography between frame 3307 and frame 4319.
Not enough matching points for homography between frame 3307 and frame 4320.
Not enough matching points for homography between frame 3307 and frame 4321.
Not enough matching points for homography between frame 3307 and frame 4322.

Finding the best anchors took 570.2293202877045
Not enough matching points for homography between frame nan and frame 3568.
Not enough matching points for homography between frame nan and frame 3580.
Not enough matching points for homography between frame nan and frame 3582.
Not enough matching points for homography between frame nan and frame 3583.
Not enough matching points for homography between frame nan and frame 3590.
Not enough matching points for homography between frame nan and frame 3591.
Not enough matching points for homography between frame nan and frame 3592.
Not enough matching points for homography between frame nan and frame 3593.
Not enough matching points for homography between frame nan and frame 3594.
Not enough matching points for homography between frame nan and frame 3595.
Not enough matching points for homography between frame nan and frame 3596.
Not enough matching points for homography between frame nan and frame 3597.
Not enough matching points for homograph

...

Not enough matching points for homography between frame nan and frame 5274.
Not enough matching points for homography between frame nan and frame 5275.
Not enough matching points for homography between frame nan and frame 5276.
Not enough matching points for homography between frame nan and frame 5277.
Not enough matching points for homography between frame nan and frame 5278.
Not enough matching points for homography between frame nan and frame 5279.
Not enough matching points for homography between frame nan and frame 5280.
Not enough matching points for homography between frame nan and frame 5281.
Not enough matching points for homography between frame nan and frame 5282.
Not enough matching points for homography between frame nan and frame 5283.
Not enough matching points for homography between frame nan and frame 5284.
Not enough matching points for homography between frame nan and frame 5285.
Not enough matching points for homography between frame nan and frame 5286.
Not enough m

Not enough matching points for homography between frame nan and frame 5486.
Not enough matching points for homography between frame nan and frame 5487.
Not enough matching points for homography between frame nan and frame 5488.
Not enough matching points for homography between frame nan and frame 5489.
Not enough matching points for homography between frame nan and frame 5490.
Not enough matching points for homography between frame nan and frame 5491.
Not enough matching points for homography between frame nan and frame 5492.
Not enough matching points for homography between frame nan and frame 5493.
Not enough matching points for homography between frame nan and frame 5494.
Not enough matching points for homography between frame nan and frame 5495.
Not enough matching points for homography between frame nan and frame 5496.
Not enough matching points for homography between frame nan and frame 5497.
Not enough matching points for homography between frame nan and frame 5498.
Not enough m

Not enough matching points for homography between frame nan and frame 5698.
Not enough matching points for homography between frame nan and frame 5699.
Not enough matching points for homography between frame nan and frame 5700.
Not enough matching points for homography between frame nan and frame 5701.
Not enough matching points for homography between frame nan and frame 5702.
Not enough matching points for homography between frame nan and frame 5703.
Not enough matching points for homography between frame nan and frame 5704.
Not enough matching points for homography between frame nan and frame 5705.
Not enough matching points for homography between frame nan and frame 5706.
Not enough matching points for homography between frame nan and frame 5707.
Not enough matching points for homography between frame nan and frame 5708.
Not enough matching points for homography between frame nan and frame 5709.
Not enough matching points for homography between frame nan and frame 5710.
Not enough m

Initial identification of anchors takes  11.335169792175293
Finding the best anchors took 70.91650986671448
Applying homography to the best anchors takes  83.730397939682
Initial identification of anchors takes  11.2866849899292
Finding the best anchors took 89.6330931186676
Applying homography to the best anchors takes  78.02314019203186
Initial identification of anchors takes  11.568897724151611
Finding the best anchors took 95.0331060886383
Applying homography to the best anchors takes  84.30274319648743
Initial identification of anchors takes  11.195688962936401
Finding the best anchors took 58.69648289680481
Applying homography to the best anchors takes  85.69877600669861
Initial identification of anchors takes  5.240554094314575
Finding the best anchors took 21.894607305526733
Applying homography to the best anchors takes  29.37653613090515
20230305 SM_Lek1 P1D1
Initial identification of anchors takes  17.547444105148315
Finding the best anchors took 431.70785570144653
Applying h

Initial identification of anchors takes  17.560019969940186
Finding the best anchors took 101.22391605377197
Applying homography to the best anchors takes  154.25699186325073
Initial identification of anchors takes  15.338137149810791
Finding the best anchors took 92.86998510360718
Applying homography to the best anchors takes  143.50082087516785
Initial identification of anchors takes  15.235223770141602
Finding the best anchors took 362.1827609539032
Applying homography to the best anchors takes  143.16257214546204
Initial identification of anchors takes  17.372829914093018
Finding the best anchors took 112.48954582214355
Applying homography to the best anchors takes  149.62756514549255
Initial identification of anchors takes  5.130162715911865
Finding the best anchors took 8.341223001480103
Applying homography to the best anchors takes  26.88859796524048
20230305 SM_Lek1 P2D3
Initial identification of anchors takes  13.863101243972778
Finding the best anchors took 97.266685962677
Ap

In [ ]:
mean_rms_error = sum(rms_errors.values()) / len(rms_errors)


fig,(ax1,ax2,ax3) = plt.subplots(1,3, figsize=(15,5))

ax1.scatter(df['frame'], df['idx'], c=df['idx'], cmap='Set1', s=5, alpha=0.4)

ax2.scatter(df['frame'], df['best_anchor_frame'], s=5)
ax2.plot(np.linspace(0,df['frame'].max()), np.linspace(0,df['frame'].max()), c='black')

sns.kdeplot(rms_errors, fill=True, ax=ax3)
ax3.vlines(mean_rms_error,0,0.32,color='black',linestyles='dashed')
ax3.set_xlim(0,)
ax3.set_ylim(0,0.32)

plt.show()